In [73]:
import numpy
import pandas as pd
import math
import numpy as np
from itertools import combinations

In [74]:
df_x = pd.read_csv('df_x.csv')
df_x = df_x.drop('Unnamed: 0', axis=1)

df_y = pd.read_csv('df_y.csv')
df_y = df_y.drop('Unnamed: 0', axis=1)

In [75]:
'''Transforma os 32 pontos em 13 pontos considerados mais "relevantes", desse modo, pontos que são da cabeça por exemplo, serão tomados a média e transformados em um único ponto.'''

def transform_body_data(df_old):
    #cabeca = 0 .. 10
    #mao esquerda = 16,18,20,22
    #mao direita  = 21,19,15,17
    #pe esquerdo  = 28, 30 , 32
    #pe direito   = 27, 29, 31

    cabeca = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','10']
    mao_esquerda = ['16', '18', '20', '22']
    mao_direita  = ['21', '19', '15', '17']
    pe_esquerdo  = ['28', '30', '32']
    pe_direito   = ['27', '29', '31']

    df_new = df_old.copy()
    for body in [cabeca,mao_esquerda,mao_direita,pe_esquerdo,pe_direito]:
        df_new[body[0]] = df_new[body].mean(axis=1)
        df_new.drop(columns=body[1:],inplace=True)
    return df_new


In [76]:
'''Faz a interpolação das linhas nulas, considerando o valor não nulo mais próximo'''

def fill_nearest(series):
    filled_series = series.copy()
    for i in range(len(series)):
        if pd.isna(series[i]):
            above = series[:i].dropna()
            below = series[i+1:].dropna()
            if not above.empty and not below.empty:
                # Obter o valor mais próximo (tanto acima quanto abaixo)
                above_dist = i - above.index[-1]
                below_dist = below.index[0] - i
                if above_dist <= below_dist:
                    filled_series[i] = above.iloc[-1]
                else:
                    filled_series[i] = below.iloc[0]
            elif not above.empty:
                filled_series[i] = above.iloc[-1]
            elif not below.empty:
                filled_series[i] = below.iloc[0]
    return filled_series

In [77]:
'''O código de interpolação assume que entre dois exercícios distintos há descanso, e que antes do ínicio do primeiro exercício também. Além disso, espera o formato gerado pelo gera_data'''

def interpola_label(df):
    exercises_passados = []
    stages_passados = []
    ultimo_exercicio = ''  # Último exercício não nulo
    alterou = False
    i = -1

    for exercise, stage in zip(df['exercise'], df['stage']):
        i += 1

        if alterou:
            alterou = False
            continue

        exercises_passados.append(exercise)
        stages_passados.append(stage)

        if type(exercise) != str:  # Se o exercício for nulo
            if ultimo_exercicio == '':  # Caso estejamos antes do primeiro exercício, entendemos que há descanso
                exercises_passados[-1] = 'descanso'
                stages_passados[-1] = -1
        else:  # Se o exercício não for nulo, atualiza o último exercício
            if ultimo_exercicio != '' and exercise != ultimo_exercicio:
                ultimo_exercicio = exercise

                exercises_passados.pop(-1)
                stages_passados.pop(-1)

                exercises_passados.append('descanso')
                stages_passados.append(-1)

                exercises_passados.append(exercise)
                stages_passados.append(1)
                
                alterou = True

                exercises_passados[i_ultimo_exercicio + 1] = 'descanso'
                stages_passados[i_ultimo_exercicio + 1] = -1

            ultimo_exercicio = exercise

            i_ultimo_exercicio = i


    df['exercise'] = exercises_passados
    df['stage'] = stages_passados

    df['exercise'] = fill_nearest(df['exercise'])
    df['stage'] = fill_nearest(df['stage'])

    return df

In [78]:
df_y = transform_body_data(df_y)
df_x = transform_body_data(df_x)

In [79]:
df_x.columns

Index(['0', '11', '12', '13', '14', '16', '21', '23', '24', '25', '26', '27',
       '28', 'ts', 'autor', 'video', 'frame', 'exercise', 'stage'],
      dtype='object')

In [80]:
def calc_points(cols, df_x, df_y, frame):
    df_x_ = df_x[df_x['frame'] == frame]
    df_y_ = df_y[df_y['frame'] == frame]

    # Extrair as coordenadas dos pontos
    px1, px2, px3 = df_x_[cols[0]].values[0], df_x_[cols[1]].values[0], df_x_[cols[2]].values[0]
    py1, py2, py3 = df_y_[cols[0]].values[0], df_y_[cols[1]].values[0], df_y_[cols[2]].values[0]

    # Criar os pontos
    p0 = np.array([px1, py1])
    p1 = np.array([px2, py2])
    p2 = np.array([px3, py3])

    # Calcular os vetores
    v0 = p0 - p1
    v2 = p2 - p1

    # Calcular o produto escalar e as normas dos vetores
    dot_product = np.dot(v0, v2)
    norm_v0 = np.linalg.norm(v0)
    norm_v2 = np.linalg.norm(v2)

    # Calcular o ângulo em radianos
    angle_rad = np.arccos(dot_product / (norm_v0 * norm_v2))

    # Converter para graus
    angle_deg = np.degrees(angle_rad)

    return angle_deg

def adiciona_empeoudeitado(linha):
    
    maximo_x = linha['maximo_x']
    minimo_x = linha['minimo_x']

    maximo_y = linha['maximo_y']
    minimo_y = linha['minimo_y']
    
    diff_x = np.abs(maximo_x - minimo_x)
    diff_y = np.abs(maximo_y - minimo_y)

    if diff_x > diff_y:
        result = 0  # Deitado
    else:
        result = 1  # Em pé

    return result
        

def calculate_angles_for_combinations(df_x, df_y, combinations):
    # Cria um DataFrame com os frames únicos
    df_angles = pd.DataFrame({'frame': df_x['frame']})

    empeoudeitado = []
    fds = []
    fds2 = []

    for comb in combinations:
        col_name = f'angle_{comb[0]}_{comb[1]}_{comb[2]}'
        angles = []

        for frame in df_angles['frame']:
            angle = calc_points(comb, df_x, df_y, frame)
            angles.append(angle)

        df_angles[col_name] = angles
    
    df_angles['ts'] = df_x['ts']
    df_angles['autor'] = df_x['autor']
    df_angles['video'] = df_x['video'] 
    df_angles['frame'] = df_x['frame']
    df_angles['exercise'] = df_x['exercise']
    df_angles['stage'] = df_x['stage']

    df_angles['maximo_x'] = df_x['0']
    df_angles['maximo_y'] = df_y['0']

    df_angles['minimo_x'] = df_x['28']
    df_angles['minimo_y'] = df_y['28']

    return df_angles

In [81]:
combinations = [['24','26','28'], ['0','24','26'], ['0','24','28'], ['16','12','24'], ['11', '13', '23']]

df = calculate_angles_for_combinations(df_x, df_y, combinations)

In [82]:
df

,frame,angle_24_26_28,angle_0_24_26,angle_0_24_28,angle_16_12_24,angle_11_13_23,ts,autor,video,exercise,stage,maximo_x,maximo_y,minimo_x,minimo_y
0,1,144.796726,138.534353,156.131421,34.065818,111.407888,0.462,Thiago (Caesar),videothiago2.mp4,NaN,NaN,0.588976,0.293952,1.211445,0.015319
1,2,91.239493,116.119356,119.353055,94.906778,50.605399,0.528,Thiago (Caesar),videothiago2.mp4,NaN,NaN,0.654749,0.384159,0.969849,0.162465
2,3,42.639997,114.716382,119.871695,22.592324,79.334341,0.561,Thiago (Caesar),videothiago2.mp4,NaN,NaN,0.677864,0.379345,0.953099,0.158521
3,4,97.938654,171.736279,157.934153,65.280905,102.130209,0.792,Thiago (Caesar),videothiago2.mp4,NaN,NaN,0.678928,0.277133,0.962275,0.068725
4,5,160.420904,166.272131,174.708328,66.520817,106.416600,0.825,Thiago (Caesar),videothiago2.mp4,NaN,NaN,0.674399,0.281127,1.000503,-0.026283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,1397,179.143522,176.673282,176.164785,0.321477,153.482218,46.563,Artur (Maurice),video2.mp4,NaN,NaN,0.359449,0.172205,0.423251,0.664449
21447,1398,175.590161,172.324217,174.814077,10.586572,170.300764,46.596,Artur (Maurice),video2.mp4,NaN,NaN,0.366996,0.170165,0.423556,0.664960
21448,1399,174.242858,171.754620,174.975332,12.274728,163.564921,46.629,Artur (Maurice),video2.mp4,NaN,NaN,0.372796,0.170692,0.430276,0.665709
21449,1400,174.415596,172.292725,175.345209,2.540010,164.127268,46.662,Artur (Maurice),video2.mp4,NaN,NaN,0.374077,0.170974,0.427355,0.666722


In [83]:
df['empeoudeitado'] = df.apply(adiciona_empeoudeitado, axis=1)

In [84]:
def row(linha):
    print(linha['angle_24_26_28'])

df.apply(row,axis=1)


144.79672617457663
91.23949317546443
42.63999660829452
97.93865408545115
160.4209041516963
167.98829199621693
150.2926986728745
162.83548572794228
164.52351134398864
98.23188178234221
160.76585512220265
168.39313218069688
168.73307382266043
137.67568692893755
157.29808559623717
178.22041457607142
154.76603538736651
147.66148487218578
148.95221593883826
161.67762054763315
145.25223039329654
155.2981256305157
166.5651532060314
177.72481315959914
177.99283470538526
174.00677445024965
179.55495431382684
173.85397836907242
179.8193218681484
179.32027566126638
178.3645432790852
176.84643416828493
177.80720107372287
160.9136726450556
162.85255591919974
143.5259159179847
152.0247285862081
128.14334656111558
162.3427902261007
170.14011755216393
169.2981468679048
162.40717799094443
153.9191057318267
146.46170499057217
148.22764926600763
159.55089687778937
151.07255402170412
154.56690724934373
147.9046098582524
150.63416857122417
153.48733501818842
145.94686359891844
145.90615447715797
147.378538

0        None
1        None
2        None
3        None
4        None
         ... 
21446    None
21447    None
21448    None
21449    None
21450    None
Length: 21451, dtype: object

In [86]:
df.loc[df["frame"] == '1401']

,frame,angle_24_26_28,angle_0_24_26,angle_0_24_28,angle_16_12_24,angle_11_13_23,ts,autor,video,exercise,stage,maximo_x,maximo_y,minimo_x,minimo_y,empeoudeitado


In [87]:
df = interpola_label(df)

In [106]:
df

,frame,angle_24_26_28,angle_0_24_26,angle_0_24_28,angle_16_12_24,angle_11_13_23,ts,autor,video,exercise,stage,maximo_x,maximo_y,minimo_x,minimo_y,empeoudeitado
0,1,144.796726,138.534353,156.131421,34.065818,111.407888,0.462,Thiago (Caesar),videothiago2.mp4,descanso,-1.0,0.588976,0.293952,1.211445,0.015319,0
1,2,91.239493,116.119356,119.353055,94.906778,50.605399,0.528,Thiago (Caesar),videothiago2.mp4,descanso,-1.0,0.654749,0.384159,0.969849,0.162465,0
2,3,42.639997,114.716382,119.871695,22.592324,79.334341,0.561,Thiago (Caesar),videothiago2.mp4,descanso,-1.0,0.677864,0.379345,0.953099,0.158521,0
3,4,97.938654,171.736279,157.934153,65.280905,102.130209,0.792,Thiago (Caesar),videothiago2.mp4,descanso,-1.0,0.678928,0.277133,0.962275,0.068725,0
4,5,160.420904,166.272131,174.708328,66.520817,106.416600,0.825,Thiago (Caesar),videothiago2.mp4,descanso,-1.0,0.674399,0.281127,1.000503,-0.026283,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,1397,179.143522,176.673282,176.164785,0.321477,153.482218,46.563,Artur (Maurice),video2.mp4,Agachamento,2.0,0.359449,0.172205,0.423251,0.664449,1
21447,1398,175.590161,172.324217,174.814077,10.586572,170.300764,46.596,Artur (Maurice),video2.mp4,Agachamento,2.0,0.366996,0.170165,0.423556,0.664960,1
21448,1399,174.242858,171.754620,174.975332,12.274728,163.564921,46.629,Artur (Maurice),video2.mp4,Agachamento,2.0,0.372796,0.170692,0.430276,0.665709,1
21449,1400,174.415596,172.292725,175.345209,2.540010,164.127268,46.662,Artur (Maurice),video2.mp4,Agachamento,2.0,0.374077,0.170974,0.427355,0.666722,1


In [107]:
df.to_csv('data_thiago.csv', index=False)